
## Obiettivo:
✔️ Capire la struttura dei dati  
✔️ Adattare il codice per il record linkage in base al nostro schema  

### 📌 Analisi dello Schema Mediato (final_mediated_schema.csv)
Il dataset contiene informazioni su aziende, con un totale di 22 colonne. Alcune delle più rilevanti per il record linkage sono:

| Colonna              | Descrizione                                                                 |
|----------------------|-----------------------------------------------------------------------------|
| company_name         | Nome dell'azienda (🔑 molto importante per il matching)                    
| industry             | Settore dell'azienda                                                        
| headquarters_city    | Città della sede (🔑 può essere usata per il blocking)                     
| headquarters_country | Paese della sede                                                            
| year_founded         | Anno di fondazione (🔑 potrebbe essere usato per similarità numerica)       
| ownership            | Tipo di proprietà (Public, Private, ecc.)                                   
| company_number       | Numero identificativo (se presente, è un match perfetto)                    
| employee_count       | Numero di dipendenti (🔑 buono per similarità numerica)                     
| market_cap_usd       | Capitalizzazione di mercato                                                 
| company_website      | Sito web (🔑 se presente, può essere un forte indicatore di match)         
| social_media_links   | Link ai social media                                                        
| representative_name  | Nome del rappresentante (può aiutare nel matching)                          
| _source_table        | Fonte del dato (utile per sapere da dove provengono i dati).                

### 📌 Funzionamento dello script
Lo script implementa un Record Linkage Pipeline, ovvero un processo per trovare e abbinare aziende simili all'interno di un dataset.

💡 **Obiettivo:** Identificare duplicati o entità corrispondenti tra i record aziendali.

#### 🔹 Step 1: Caricamento e Pulizia del Dataset
- Carica il dataset CSV `final_mediated_schema.csv`.
- Seleziona solo le colonne utili (`company_name`, `industry`, `headquarters_city`, `year_founded`, ecc.).
- Rimuove le righe senza `company_name`, perché il nome dell'azienda è essenziale per il confronto.
- Converte `year_founded` e `employee_count` in numeri, evitando errori durante il matching.

#### 🔹 Step 2: Blocking per Ridurre i Confronti
Il blocking è una tecnica per ridurre il numero di confronti inutili tra le aziende. Lo script utilizza tre strategie di blocking:
- **Blocking su `headquarters_city` e `industry`** → Considera solo aziende con la stessa sede e settore.
- **Sorted Neighbourhood su `company_name`** → Confronta aziende con nomi simili entro una finestra di 5 righe.
- **Blocking su `ownership`** → Confronta solo aziende con la stessa tipologia di proprietà.

💡 **Risultato:** Il numero di confronti si riduce da milioni a un sottoinsieme più piccolo di `candidate_pairs`.

#### 🔹 Step 3: Pairwise Matching (Confronto tra Coppie)
Una volta ottenute le coppie candidate, lo script calcola quanto sono simili usando diverse metriche:

| Campo           | Metodo       | Descrizione                                                                 |
|-----------------|--------------|-----------------------------------------------------------------------------|
| `company_name`  | Jaro-Winkler | Confronta i nomi in base alla distanza tra i caratteri (similitudine testuale). |
| `company_website` | Exact Match  | Controlla se i siti web delle aziende coincidono esattamente.                |
| `year_founded`  | Gaussiano    | Permette piccole variazioni nell'anno di fondazione.                         |
| `employee_count`| Gaussiano    | Permette variazioni nel numero di dipendenti, considerando una distribuzione statistica. |

💡 **Risultato:** Viene creata una matrice di similarità tra i record aziendali.

#### 🔹 Step 4: Filtraggio delle Migliori Corrispondenze
Lo script seleziona le migliori corrispondenze utilizzando una soglia di similarità: ✅ Le aziende sono considerate matching se:
- `company_name` ha una somiglianza > 85% (Jaro-Winkler).
- `company_website` è esattamente uguale.
- `year_founded` e `employee_count` hanno una similitudine combinata > 80%.

💡 **Risultato:** Si ottiene un sottoinsieme di aziende che probabilmente rappresentano la stessa entità.

#### 🔹 Step 5: Salvataggio dei Risultati
Infine, lo script:
- Salva le corrispondenze trovate in un file CSV: `matched_companies.csv`
- Mostra il numero di aziende abbinate.
````

In [1]:
import pandas as pd
import recordlinkage

# 📂 Caricare il dataset con lo schema mediato
schema_file = "main_outputs/final_mediated_schema.csv"  # Modifica con il percorso corretto
df = pd.read_csv(schema_file)

# 🔎 Selezioniamo solo le colonne utili per il matching
df = df[['company_name', 'industry', 'headquarters_city', 'year_founded', 
         'ownership', 'company_number', 'employee_count', 'company_website']]

# 📌 Rimuoviamo righe con `company_name` mancante
df = df.dropna(subset=['company_name'])

# ✅ Convertiamo i numeri nel formato corretto
df['year_founded'] = pd.to_numeric(df['year_founded'], errors='coerce')
df['employee_count'] = pd.to_numeric(df['employee_count'], errors='coerce')

# 🔎 Controlliamo i NaN
print(df[['year_founded', 'employee_count']].isna().sum())

# ✅ STEP 1: Blocking per ridurre il numero di confronti
indexer = recordlinkage.Index()

# 🔹 **Blocking 1: Exact Matching su `headquarters_city` e `industry`**
indexer.add(recordlinkage.index.Block(['headquarters_city', 'industry']))

# 🔹 **Blocking 2: Sorted Neighbourhood su `company_name` con finestra 5**
indexer.add(recordlinkage.index.SortedNeighbourhood('company_name', window=5))

# 🔹 **Blocking 3: Exact Matching su `ownership`**
indexer.add(recordlinkage.index.Block(['ownership']))

# Creiamo le coppie candidate
candidate_pairs = indexer.index(df)

print(f"🔍 Generato {len(candidate_pairs)} confronti dopo il blocking.")

# ✅ STEP 2: Pairwise Matching (Similarità tra i record)
compare = recordlinkage.Compare()

# 📌 Matching su `company_name` con Jaro-Winkler (string similarity)
compare.string('company_name', 'company_name', method='jarowinkler', label='name_sim')

# 📌 Matching su `company_website` (valore esatto, se presente)
compare.exact('company_website', 'company_website', label='website_exact')

# 📌 Matching su `year_founded` con tolleranza numerica (metodo Gaussiano)
compare.numeric('year_founded', 'year_founded', method='gauss', scale=5, label='year_sim')

# 📌 Matching su `employee_count` con metodo "gauss"
compare.numeric('employee_count', 'employee_count', method='gauss', scale=5, label='employees_sim')  # 🔹 Prova anche step

# ✅ STEP 3: Creazione della matrice di similarità tra i record
similarity_matrix = compare.compute(candidate_pairs, df)

# ✅ STEP 4: Filtraggio delle migliori corrispondenze
# Usiamo una soglia: match con almeno 2 metriche alte
matches = similarity_matrix[(similarity_matrix['name_sim'] > 0.85) | 
                            (similarity_matrix['website_exact'] == 1) | 
                            ((similarity_matrix['year_sim'] > 0.8) & (similarity_matrix['employees_sim'] > 0.8))]

# ✅ STEP 5: Salviamo i risultati
matches.to_csv("matched_companies.csv")
print(f"✅ Matching completato! Salvato in 'matched_companies.csv'.")


C:\Users\luca-\AppData\Local\Temp\ipykernel_36996\2829431938.py:6: DtypeWarning: Columns (2,5,6,7,9,14,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(schema_file)


year_founded      55642
employee_count    64309
dtype: int64
🔍 Generato 21304659 confronti dopo il blocking.
✅ Matching completato! Salvato in 'matched_companies.csv'.


## 📌 Passaggio 1: Convertire ground_truth.json in CSV

Attualmente il file JSON ha questa struttura:

```json
{
    "ground_truth": [
        {
            "1&1": "companiesMarketCap_dataset",
            "1&1 AG": "disfold-com"
        },
        {
            "AMPLUS": "ft-com",
            "AMPLUS SOLAR": "AmbitionBox"
        }
    ]
}
```

Dobbiamo convertirlo in un formato compatibile con il CSV che abbiamo creato. Un buon formato potrebbe essere:

```csv
company_a,source_a,company_b,source_b
1&1,companiesMarketCap_dataset,1&1 AG,disfold-com
AMPLUS,ft-com,AMPLUS SOLAR,AmbitionBox
```

## Preparare i Dati per Ditto

Ditto richiede un dataset in formato TSV con questa struttura:

```
sentence_1 \t sentence_2 \t label
```

Esempio:

```
1&1 è una società di telecomunicazioni. \t 1&1 AG è un operatore tedesco di telecomunicazioni. \t 1
AMPLUS è una società di energia. \t AMPLUS SOLAR è una società di energia rinnovabile. \t 0
```

📌 Script per convertire `ground_truth.csv` e `matched_companies.csv` in TSV per Ditto:

In [5]:
import pandas as pd

# 📂 Carichiamo i dati originali e la ground truth
df_gt = pd.read_csv("data/ground_truth.csv")  # Ground-Truth
df_matched = pd.read_csv("matched_companies.csv")  # RecordLinkage
df_original = pd.read_csv("main_outputs/final_mediated_schema.csv")  # Dataset originale

# 🔍 Controlliamo gli indici originali
print("Colonne in matched_companies:", df_matched.columns.tolist())
print("Colonne in final_mediated_schema:", df_original.columns.tolist())

# 🛠 Ricolleghiamo gli indici di matched_companies.csv ai nomi reali delle aziende
df_matched = df_matched.rename(columns={'Unnamed: 0': 'index_1', 'Unnamed: 1': 'index_2'})

# Convertiamo gli indici a numeri interi
df_matched[['index_1', 'index_2']] = df_matched[['index_1', 'index_2']].astype(int)

# Facciamo il merge per recuperare i nomi reali delle aziende
df_matched = df_matched.merge(df_original[['company_name', 'industry', 'employee_count']], 
                              left_on='index_1', right_index=True, how='left')
df_matched = df_matched.merge(df_original[['company_name']], 
                              left_on='index_2', right_index=True, how='left', suffixes=('_1', '_2'))

# 📌 Prepara il dataset per Ditto (Ground-Truth)
gt_pairs = []
for _, row in df_gt.iterrows():
    sentence_1 = f"{row['company_a']} è una società con sede in {row['source_a']}."
    sentence_2 = f"{row['company_b']} è una società con sede in {row['source_b']}."
    gt_pairs.append(f"{sentence_1}\t{sentence_2}\t1")

# 📌 Prepara il dataset per Ditto (RecordLinkage Matches)
rl_pairs = []
for _, row in df_matched.iterrows():
    sentence_1 = f"{row['company_name_1']} è un'azienda in {row['industry']}."
    sentence_2 = f"{row['company_name_2']} è un'azienda con {row['employee_count']} dipendenti."
    rl_pairs.append(f"{sentence_1}\t{sentence_2}\t0")

# 📝 Salviamo i file in formato TSV con UTF-8 per evitare errori di encoding
with open("ditto/data/ditto_train.tsv", "w", encoding="utf-8") as f:
    f.write("\n".join(gt_pairs))

with open("ditto/data/ditto_test.tsv", "w", encoding="utf-8") as f:
    f.write("\n".join(rl_pairs))

print("✅ File TSV preparati per Ditto!")

Colonne in matched_companies: ['Unnamed: 0', 'Unnamed: 1', 'name_sim', 'website_exact', 'year_sim', 'employees_sim']
Colonne in final_mediated_schema: ['company_name', 'industry', 'headquarters_address', 'headquarters_city', 'headquarters_country', 'year_founded', 'ownership', 'company_number', 'employee_count', 'market_cap_usd', 'total_revenue_usd', 'net_profit_usd', 'total_assets_usd', 'company_website', 'social_media_links', 'representative_name', 'total_raised', 'company_description', 'company_stage', 'share_price', 'legal_form', '_source_table']
✅ File TSV preparati per Ditto!


In [1]:
import torch
import transformers
import spacy

print("PyTorch version:", torch.__version__)
print("Transformers version:", transformers.__version__)
print("spaCy version:", spacy.__version__)


PyTorch version: 1.9.0+cu111
Transformers version: 4.9.2
spaCy version: 3.1.0


## 🔹 Step 2: Installare e Configurare Ditto

Esegui questi comandi per installare Ditto:

```bash
git clone https://github.com/megagonlabs/ditto.git
cd ditto
pip install -r requirements.txt
```

## 🔹 Step 3: Addestrare Ditto

Ditto utilizza modelli Transformer (BERT, RoBERTa). Esegui questo script per addestrarlo sulla Ground-Truth:

```bash
python train_ditto.py \
    --task custom \
    --trainset data/ditto_train.tsv \
    --devset data/ditto_train.tsv \
    --batch_size 16 \
    --max_len 64 \
    --epochs 5 \
    --lr 3e-5 \
    --save_model data/ditto_model.pt
```

🔹 **Risultato:** Modello salvato in `data/ditto_model.pt`.

## 🔹 Step 4: Applicare Ditto al Pairwise Matching

Ora che abbiamo addestrato Ditto, usiamo il modello per fare previsioni sui nostri dati:

```bash
python predict_ditto.py \
    --task custom \
    --testset data/ditto_test.tsv \
    --model data/ditto_model.pt \
    --output data/ditto_results.tsv
```

🔹 **Risultato:** Previsioni salvate in `data/ditto_results.tsv`.

## Confrontare i Risultati tra RecordLinkage e Ditto

Ora confrontiamo le prestazioni di RecordLinkage vs Ditto.

📌 Script per calcolare precision, recall e F1-score:

In [ ]:
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support

# 📂 Carichiamo i risultati
df_rl = pd.read_csv("matched_companies.csv")  # RecordLinkage
df_ditto = pd.read_csv("data/ditto_results.tsv", sep="\t", header=None, names=["sentence_1", "sentence_2", "label"])
df_gt = pd.read_csv("data/ground_truth.csv")  # Ground-Truth

# 🔄 Convertiamo in set per il confronto
set_rl = set(df_rl["company_name"])
set_ditto = set(df_ditto[df_ditto["label"] == 1]["sentence_1"])
set_gt = set(df_gt["company_a"])

# 📊 Calcoliamo Precision, Recall e F1-score
precision_rl, recall_rl, f1_rl, _ = precision_recall_fscore_support(
    [1 if x in set_gt else 0 for x in set_rl],
    [1 for _ in set_rl], average='binary')

precision_ditto, recall_ditto, f1_ditto, _ = precision_recall_fscore_support(
    [1 if x in set_gt else 0 for x in set_ditto],
    [1 for _ in set_ditto], average='binary')

# 📌 Creiamo il report
comparison = pd.DataFrame({
    "Metodo": ["RecordLinkage", "Ditto"],
    "Precision": [precision_rl, precision_ditto],
    "Recall": [recall_rl, recall_ditto],
    "F1-Score": [f1_rl, f1_ditto]
})

# 📝 Salviamo il confronto
comparison.to_csv("comparison_report.csv", index=False)
print("✅ Confronto completato! Salvato in 'comparison_report.csv'.")
